# **Data Collecting**

## **NOTE**

This script collects *publicly available* product data from **tiki.vn**,
specifically from category IDs: 
- 8371: Đồng hồ và trang sức
- 1815: Thiết bị số - Phụ kiện số
- 915: Thời trang nam
- 931: Thời trang nữ

Only general product information is collected — including **images, names,
and descriptions** — to serve **educational and demo purposes only**.

⚠️ No personal or user-related data is collected, stored, or used.
The dataset is solely intended for learning, research, and
demonstrating product display functionality on our website.

### **Import libraries**

In [2]:
import requests
import time
import csv
import cloudinary
import cloudinary.uploader
import os

cloudinary.config(
  cloud_name = "dvxmaiofh", 
  api_key = "834668983718514", 
  api_secret = "KI_mVAdKhFNvtbJN4w9TWSzJqno",
  secure = True
)

### **Variable Cofig**

In [9]:
CATEGORY_ID_LIST = [15320] 
MAX_PRODUCTS = 40

headers = {
    "User-Agent": "Mozilla/5.0"
}

os.makedirs("data", exist_ok=True)

### **Collecting data**

In [10]:
all_categories = []

for CATEGORY_ID in CATEGORY_ID_LIST:
    items = []
    page = 1

    # Lấy thông tin Category
    try:
        cat_info = requests.get(f"https://tiki.vn/api/v2/categories/{CATEGORY_ID}", headers=headers).json()
        category_name = cat_info.get("name", f"Category_{CATEGORY_ID}")
    except Exception as e:
        print(f"Lỗi lấy thông tin category {CATEGORY_ID}: {e}")
        continue

    print(f"Collecting products from category: {category_name} (ID: {CATEGORY_ID})")
    
    while len(items) < MAX_PRODUCTS:
        LIST_API = f"https://tiki.vn/api/v2/products?category={CATEGORY_ID}&page={page}"
        try:
            resp = requests.get(LIST_API, headers=headers).json()
            data = resp.get("data", [])
        except Exception as e:
            print(f"Lỗi lấy danh sách sản phẩm trang {page}: {e}")
            break

        if not data:
            break

        for p in data:
            if len(items) >= MAX_PRODUCTS:
                break

            pid = p["id"]
            try:
                detail = requests.get(f"https://tiki.vn/api/v2/products/{pid}", headers=headers).json()
            except Exception as e:
                print(f"Lỗi lấy chi tiết sản phẩm {pid}: {e}")
                continue

            desc = detail.get("description", "")

            # Lấy link gốc từ Tiki (tối đa 5 ảnh)
            raw_image_list = [img["base_url"] for img in detail.get("images", []) if "base_url" in img][:5]
            
            # --- [MỚI] KIỂM TRA SỐ LƯỢNG ẢNH ---
            if len(raw_image_list) < 3:
                # Nếu ít hơn 3 ảnh thì bỏ qua, không cần fallback thumbnail nữa vì đằng nào cũng skip
                print(f"   [SKIP] SP {pid} bị loại vì chỉ có {len(raw_image_list)} ảnh (< 3).")
                continue
            # -----------------------------------

            # ================= XỬ LÝ UPLOAD CLOUDINARY =================
            cloudinary_urls = []
            print(f"   >>> Đang upload {len(raw_image_list)} ảnh lên Cloudinary cho SP: {pid}...")
            
            for img_url in raw_image_list:
                if img_url:
                    try:
                        upload_result = cloudinary.uploader.upload(img_url, folder="SnapBid")
                        cloudinary_urls.append(upload_result['secure_url'])
                    except Exception as e:
                        print(f"      [Lỗi Upload] Không thể up ảnh {img_url}: {e}")
                        cloudinary_urls.append(img_url)
            
            # Kiểm tra lại lần nữa sau khi upload (đề phòng upload lỗi hết sạch)
            if len(cloudinary_urls) < 3:
                 print(f"   [SKIP] SP {pid} bị loại sau khi upload vì chỉ còn {len(cloudinary_urls)} ảnh thành công.")
                 continue
            # ===========================================================

            items.append({
                "id": pid,
                "name": p.get("name"),
                "price": p.get("price"),
                "images": cloudinary_urls,
                "desc_html": desc
            })

            print(f"GET {len(items)}. {p.get('name')[:40]}... (Đã up {len(cloudinary_urls)} ảnh)")
            time.sleep(0.3)

        page += 1
        time.sleep(0.5)

    all_categories.append({
        "category_name": category_name,
        "products": items
    })
    
os.makedirs("data/earring", exist_ok=True)

with open("data/earring/tiki_products.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(
        f,
        fieldnames=["category_name", "id", "name", "price", "images", "desc_html"]
    )
    writer.writeheader()

    for cat in all_categories:
        category_name = cat["category_name"]
        for p in cat["products"]:
            writer.writerow({
                "category_name": category_name,
                "id": p.get("id"),
                "name": p.get("name"),
                "price": p.get("price"),
                "images": ", ".join(p.get("images", [])),
                "desc_html": p.get("desc_html", "").replace("\n", " ")
            })

   >>> Đang upload 3 ảnh lên Cloudinary cho SP: 277880948...
GET 1. Bông tai cỏ 4 lá đá ngọc bích mệnh hỏa ,... (Đã up 3 ảnh)
   >>> Đang upload 3 ảnh lên Cloudinary cho SP: 277880861...
GET 2. Bông tai cỏ 4 lá đá sapphire mệnh thủy, ... (Đã up 3 ảnh)
   >>> Đang upload 5 ảnh lên Cloudinary cho SP: 275875289...
GET 3. Hoa Tai, Bông Tai Nữ Hoa Đá Hàng Chính H... (Đã up 5 ảnh)
   >>> Đang upload 3 ảnh lên Cloudinary cho SP: 274123120...
GET 4. Bông tai bạc đính đá ngọc bích 6mm mệnh ... (Đã up 3 ảnh)
   >>> Đang upload 5 ảnh lên Cloudinary cho SP: 221512844...
GET 5. Đôi bông tai mới bông hồng nổi cao sịn đ... (Đã up 5 ảnh)
   >>> Đang upload 5 ảnh lên Cloudinary cho SP: 203950701...
GET 6. Chốt bạc cho bông tai nữ bạc 925 loại ch... (Đã up 5 ảnh)
   >>> Đang upload 3 ảnh lên Cloudinary cho SP: 196333334...
GET 7. 14K Gold Post Earrings Aquablue - MOON J... (Đã up 3 ảnh)
   >>> Đang upload 3 ảnh lên Cloudinary cho SP: 196333000...
GET 8. 14K Gold Post Earrings Violet - MOON Jew... (Đã up